# Quantum ESPRESSO Workshop - Part 7: Density of States (DOS)

## Learning Objectives
1. Understand the Density of States concept
2. Perform DOS calculation workflow (SCF → NSCF → dos.x)
3. Calculate total and projected DOS (PDOS)
4. Visualize and interpret DOS

---

## 1. What is the Density of States?

### Definition

The Density of States $g(E)$ gives the number of electronic states per unit energy:

$$g(E) = \sum_n \int_{BZ} \delta(E - \varepsilon_{n\mathbf{k}}) \frac{d\mathbf{k}}{(2\pi)^3}$$

### Physical Meaning

- **DOS at E**: How many states are available at energy E
- **Integrated DOS**: Total number of electrons
- **DOS at Fermi level**: Related to electronic specific heat, conductivity

### DOS vs Band Structure

| Band Structure | DOS |
|----------------|-----|
| E(k) dispersion | Energy distribution |
| Shows gap location | Shows gap width |
| k-resolved | k-integrated |
| Qualitative | Quantitative |

### Projected DOS (PDOS)

PDOS decomposes the total DOS by:
- **Atom**: Which atom contributes
- **Orbital**: s, p, d character

---

## 2. Setup

In [ ]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re
import time
import json

# Paths
WORKSHOP_ROOT = Path('/home/claude/qe_workshop')
PSEUDO_DIR = WORKSHOP_ROOT / 'pseudopotentials'
OUTPUT_DIR = WORKSHOP_ROOT / 'outputs'

# Working directory
WORK_DIR = OUTPUT_DIR / '07_dos'
WORK_DIR.mkdir(exist_ok=True)
(WORK_DIR / 'tmp').mkdir(exist_ok=True)

print(f"Working directory: {WORK_DIR}")

In [ ]:
# Load converged parameters
params_file = WORKSHOP_ROOT / 'converged_parameters.json'

if params_file.exists():
    with open(params_file, 'r') as f:
        params = json.load(f)
    ecutwfc = params.get('ecutwfc_recommended', 40.0)
    ecutrho_factor = params.get('ecutrho_factor', 8)
    kgrid = params.get('kpoints_recommended', 8)
    celldm1 = params.get('celldm1_optimized_bohr', 10.26)
    print("Loaded converged parameters:")
    print(f"  ecutwfc = {ecutwfc} Ry")
    print(f"  k-points = {kgrid}×{kgrid}×{kgrid}")
    print(f"  celldm(1) = {celldm1} Bohr")
else:
    ecutwfc = 40.0
    ecutrho_factor = 8
    kgrid = 8
    celldm1 = 10.26
    print("Using default parameters")

ecutrho = ecutwfc * ecutrho_factor

---

## 3. DOS Calculation Workflow

The DOS calculation requires three steps:

1. **SCF**: Self-consistent calculation
2. **NSCF**: Non-self-consistent calculation with dense k-grid and tetrahedron method
3. **dos.x**: Post-processing to compute DOS

### Step 1: SCF Calculation

In [ ]:
# Generate SCF input
scf_input = f"""&CONTROL
    calculation = 'scf'
    prefix = 'silicon'
    outdir = './tmp'
    pseudo_dir = '{PSEUDO_DIR}'
    verbosity = 'high'
/

&SYSTEM
    ibrav = 2
    celldm(1) = {celldm1}
    nat = 2
    ntyp = 1
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
/

&ELECTRONS
    conv_thr = 1.0e-8
    mixing_beta = 0.7
/

ATOMIC_SPECIES
    Si  28.0855  Si.upf

ATOMIC_POSITIONS {{crystal}}
    Si  0.00  0.00  0.00
    Si  0.25  0.25  0.25

K_POINTS {{automatic}}
    {kgrid} {kgrid} {kgrid} 0 0 0
"""

# Write and run SCF
scf_file = WORK_DIR / 'si_scf.in'
with open(scf_file, 'w') as f:
    f.write(scf_input)

print("Running SCF calculation...")
start = time.time()

result = subprocess.run(
    ['pw.x', '-in', 'si_scf.in'],
    capture_output=True,
    text=True,
    cwd=WORK_DIR,
    timeout=300
)

with open(WORK_DIR / 'si_scf.out', 'w') as f:
    f.write(result.stdout)

elapsed = time.time() - start

if 'convergence has been achieved' in result.stdout:
    print(f"✓ SCF converged in {elapsed:.2f} s")
else:
    print("✗ SCF did not converge!")

### Step 2: NSCF Calculation with Dense K-grid

For accurate DOS, we need:
- **Much denser k-grid** than SCF (e.g., 16×16×16 or more)
- **Tetrahedron method** for Brillouin zone integration

In [ ]:
# Dense k-grid for DOS
kgrid_dos = 16  # Much denser than SCF

nscf_input = f"""&CONTROL
    calculation = 'nscf'
    prefix = 'silicon'
    outdir = './tmp'
    pseudo_dir = '{PSEUDO_DIR}'
    verbosity = 'high'
/

&SYSTEM
    ibrav = 2
    celldm(1) = {celldm1}
    nat = 2
    ntyp = 1
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'tetrahedra'
    nbnd = 12
/

&ELECTRONS
    conv_thr = 1.0e-8
/

ATOMIC_SPECIES
    Si  28.0855  Si.upf

ATOMIC_POSITIONS {{crystal}}
    Si  0.00  0.00  0.00
    Si  0.25  0.25  0.25

K_POINTS {{automatic}}
    {kgrid_dos} {kgrid_dos} {kgrid_dos} 0 0 0
"""

# Write and run NSCF
nscf_file = WORK_DIR / 'si_nscf.in'
with open(nscf_file, 'w') as f:
    f.write(nscf_input)

print(f"Running NSCF calculation with {kgrid_dos}×{kgrid_dos}×{kgrid_dos} k-grid...")
start = time.time()

result = subprocess.run(
    ['pw.x', '-in', 'si_nscf.in'],
    capture_output=True,
    text=True,
    cwd=WORK_DIR,
    timeout=600
)

with open(WORK_DIR / 'si_nscf.out', 'w') as f:
    f.write(result.stdout)

elapsed = time.time() - start

if 'JOB DONE' in result.stdout:
    print(f"✓ NSCF completed in {elapsed:.2f} s")
    # Extract number of k-points
    for line in result.stdout.split('\n'):
        if 'number of k points' in line:
            print(f"  {line.strip()}")
            break
else:
    print("✗ NSCF may have failed")

### Step 3: DOS Post-processing

In [ ]:
# dos.x input
dos_input = f"""&DOS
    prefix = 'silicon'
    outdir = './tmp'
    fildos = 'si_dos.dat'
    Emin = -15.0
    Emax = 10.0
    DeltaE = 0.01
/
"""

dos_file = WORK_DIR / 'si_dos.in'
with open(dos_file, 'w') as f:
    f.write(dos_input)

print("Running dos.x...")

result = subprocess.run(
    ['dos.x', '-in', 'si_dos.in'],
    capture_output=True,
    text=True,
    cwd=WORK_DIR,
    timeout=120
)

with open(WORK_DIR / 'si_dos.out', 'w') as f:
    f.write(result.stdout)

if (WORK_DIR / 'si_dos.dat').exists():
    print("✓ DOS data file created: si_dos.dat")
    
    # Extract Fermi energy
    for line in result.stdout.split('\n'):
        if 'Fermi' in line:
            print(f"  {line.strip()}")
else:
    print("✗ DOS calculation may have failed")
    print(result.stdout[:500])

---

## 4. Projected DOS (PDOS) Calculation

In [ ]:
# projwfc.x input for PDOS
projwfc_input = f"""&PROJWFC
    prefix = 'silicon'
    outdir = './tmp'
    filpdos = 'si_pdos'
    Emin = -15.0
    Emax = 10.0
    DeltaE = 0.01
/
"""

projwfc_file = WORK_DIR / 'si_projwfc.in'
with open(projwfc_file, 'w') as f:
    f.write(projwfc_input)

print("Running projwfc.x for PDOS...")
start = time.time()

result = subprocess.run(
    ['projwfc.x', '-in', 'si_projwfc.in'],
    capture_output=True,
    text=True,
    cwd=WORK_DIR,
    timeout=300
)

with open(WORK_DIR / 'si_projwfc.out', 'w') as f:
    f.write(result.stdout)

elapsed = time.time() - start

# List PDOS files created
pdos_files = list(WORK_DIR.glob('si_pdos*'))
if pdos_files:
    print(f"✓ PDOS calculation completed in {elapsed:.2f} s")
    print(f"  Created {len(pdos_files)} PDOS files:")
    for f in sorted(pdos_files)[:10]:
        print(f"    {f.name}")
    if len(pdos_files) > 10:
        print(f"    ... and {len(pdos_files) - 10} more")
else:
    print("✗ PDOS files not found")

---

## 5. Parse and Plot DOS

In [ ]:
def parse_dos_file(filename):
    """
    Parse QE dos.x output file.
    
    Returns
    -------
    energy : array
        Energy values (eV)
    dos : array
        DOS values (states/eV)
    idos : array
        Integrated DOS
    fermi : float
        Fermi energy (eV)
    """
    energy = []
    dos = []
    idos = []
    fermi = None
    
    with open(filename, 'r') as f:
        for line in f:
            if line.startswith('#'):
                # Header line - extract Fermi energy
                if 'EFermi' in line:
                    match = re.search(r'EFermi\s*=\s*([\d.+-]+)', line)
                    if match:
                        fermi = float(match.group(1))
                continue
            
            parts = line.split()
            if len(parts) >= 2:
                energy.append(float(parts[0]))
                dos.append(float(parts[1]))
                if len(parts) >= 3:
                    idos.append(float(parts[2]))
    
    return np.array(energy), np.array(dos), np.array(idos) if idos else None, fermi


def parse_pdos_file(filename):
    """
    Parse QE projwfc.x PDOS output file.
    
    Returns
    -------
    energy : array
    ldos : array
        Local DOS
    pdos : array
        Projected DOS (by orbital)
    """
    energy = []
    ldos = []
    pdos = []
    
    with open(filename, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            parts = line.split()
            if len(parts) >= 2:
                energy.append(float(parts[0]))
                ldos.append(float(parts[1]))
                if len(parts) >= 3:
                    pdos.append(float(parts[2]))
    
    return np.array(energy), np.array(ldos), np.array(pdos) if pdos else None

In [ ]:
# Parse total DOS
dos_file = WORK_DIR / 'si_dos.dat'

if dos_file.exists():
    energy, dos, idos, fermi = parse_dos_file(dos_file)
    
    print(f"Total DOS data:")
    print(f"  Energy range: {energy.min():.2f} to {energy.max():.2f} eV")
    print(f"  Number of points: {len(energy)}")
    if fermi:
        print(f"  Fermi energy: {fermi:.4f} eV")
    if idos is not None:
        print(f"  Total integrated DOS: {idos[-1]:.2f} electrons")
else:
    print("DOS file not found!")

In [ ]:
# Plot total DOS
if dos_file.exists():
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Shift to Fermi level = 0
    e_shift = fermi if fermi else 0
    energy_shifted = energy - e_shift
    
    # Plot 1: Total DOS
    ax1 = axes[0]
    ax1.plot(energy_shifted, dos, 'b-', linewidth=1.5)
    ax1.axvline(x=0, color='r', linestyle='--', label='Fermi level')
    ax1.fill_between(energy_shifted, dos, where=(energy_shifted <= 0), 
                     alpha=0.3, color='blue', label='Occupied')
    ax1.set_xlabel('Energy (eV)', fontsize=12)
    ax1.set_ylabel('DOS (states/eV)', fontsize=12)
    ax1.set_title('Total Density of States - Silicon', fontsize=14)
    ax1.set_xlim(-15, 10)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Integrated DOS
    ax2 = axes[1]
    if idos is not None:
        ax2.plot(energy_shifted, idos, 'g-', linewidth=1.5)
        ax2.axvline(x=0, color='r', linestyle='--', label='Fermi level')
        ax2.axhline(y=8, color='orange', linestyle=':', label='8 valence electrons')
        ax2.set_xlabel('Energy (eV)', fontsize=12)
        ax2.set_ylabel('Integrated DOS (electrons)', fontsize=12)
        ax2.set_title('Integrated DOS', fontsize=14)
        ax2.set_xlim(-15, 10)
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(str(WORK_DIR / 'silicon_dos.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print(f"\nFigure saved to: {WORK_DIR / 'silicon_dos.png'}")

---

## 6. Parse and Plot PDOS

In [ ]:
# Find and parse PDOS files
# File naming: si_pdos.pdos_atm#N(Element)_wfc#M(orbital)

pdos_data = {}

# Look for atom-resolved PDOS
for pdos_file in sorted(WORK_DIR.glob('si_pdos.pdos_atm*')):
    # Extract atom and orbital info from filename
    name = pdos_file.name
    
    # Parse filename pattern: si_pdos.pdos_atm#1(Si)_wfc#1(s)
    match = re.search(r'atm#(\d+)\((\w+)\)_wfc#(\d+)\((\w+)\)', name)
    if match:
        atom_num = int(match.group(1))
        element = match.group(2)
        wfc_num = int(match.group(3))
        orbital = match.group(4)
        
        energy, ldos, pdos = parse_pdos_file(pdos_file)
        
        key = f"{element}{atom_num}_{orbital}"
        pdos_data[key] = {
            'energy': energy,
            'ldos': ldos,
            'element': element,
            'orbital': orbital,
            'atom_num': atom_num
        }

print(f"Parsed {len(pdos_data)} PDOS files:")
for key in sorted(pdos_data.keys()):
    print(f"  {key}")

In [ ]:
# Plot PDOS by orbital type
if pdos_data:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Combine PDOS by orbital type (sum over equivalent atoms)
    orbital_pdos = {}
    
    for key, data in pdos_data.items():
        orbital = data['orbital']
        if orbital not in orbital_pdos:
            orbital_pdos[orbital] = data['ldos'].copy()
        else:
            orbital_pdos[orbital] += data['ldos']
    
    # Get energy (assume all have same energy grid)
    energy = list(pdos_data.values())[0]['energy']
    e_shift = fermi if fermi else 0
    energy_shifted = energy - e_shift
    
    # Plot each orbital contribution
    colors = {'s': 'blue', 'p': 'red', 'd': 'green', 'f': 'orange'}
    
    total = np.zeros_like(energy)
    for orbital, pdos_values in sorted(orbital_pdos.items()):
        color = colors.get(orbital[0], 'gray')
        ax.fill_between(energy_shifted, pdos_values, alpha=0.5, color=color, label=f'Si {orbital}')
        ax.plot(energy_shifted, pdos_values, color=color, linewidth=1)
        total += pdos_values
    
    # Plot total for comparison (if available)
    if dos_file.exists():
        ax.plot(energy_shifted, dos, 'k-', linewidth=2, alpha=0.7, label='Total DOS')
    
    ax.axvline(x=0, color='gray', linestyle='--', alpha=0.7)
    ax.set_xlabel('Energy (eV)', fontsize=12)
    ax.set_ylabel('DOS (states/eV)', fontsize=12)
    ax.set_title('Projected Density of States - Silicon', fontsize=14)
    ax.set_xlim(-15, 10)
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(str(WORK_DIR / 'silicon_pdos.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print(f"\nFigure saved to: {WORK_DIR / 'silicon_pdos.png'}")

---

## 7. DOS Analysis

In [ ]:
# Analyze DOS features
if dos_file.exists():
    e_shift = fermi if fermi else 0
    energy_shifted = energy - e_shift
    
    # Find band gap from DOS
    # Look for region near Fermi level where DOS ≈ 0
    threshold = 0.01  # States/eV threshold for "gap"
    
    # Find VBM (last energy with DOS > threshold below Fermi)
    valence_mask = (energy_shifted <= 0) & (dos > threshold)
    if np.any(valence_mask):
        vbm_idx = np.where(valence_mask)[0][-1]
        vbm_energy = energy_shifted[vbm_idx]
    else:
        vbm_energy = None
    
    # Find CBM (first energy with DOS > threshold above Fermi)
    conduction_mask = (energy_shifted > 0) & (dos > threshold)
    if np.any(conduction_mask):
        cbm_idx = np.where(conduction_mask)[0][0]
        cbm_energy = energy_shifted[cbm_idx]
    else:
        cbm_energy = None
    
    print("DOS Analysis")
    print("=" * 50)
    if fermi:
        print(f"Fermi energy: {fermi:.4f} eV")
    if vbm_energy is not None:
        print(f"VBM (from DOS): {vbm_energy:.4f} eV (relative to E_F)")
    if cbm_energy is not None:
        print(f"CBM (from DOS): {cbm_energy:.4f} eV (relative to E_F)")
    if vbm_energy is not None and cbm_energy is not None:
        gap = cbm_energy - vbm_energy
        print(f"Band gap (from DOS): {gap:.4f} eV")
    
    # Check for states at Fermi level (metallic vs insulating)
    dos_at_fermi = np.interp(0, energy_shifted, dos)
    print(f"\nDOS at Fermi level: {dos_at_fermi:.4f} states/eV")
    if dos_at_fermi < 0.1:
        print("→ System is a SEMICONDUCTOR/INSULATOR")
    else:
        print("→ System is METALLIC")

---

## 8. Save Results

In [ ]:
# Save DOS data
if dos_file.exists():
    dos_results = {
        'energy_ev': energy.tolist(),
        'dos_states_per_ev': dos.tolist(),
        'integrated_dos': idos.tolist() if idos is not None else None,
        'fermi_energy_ev': fermi,
        'kgrid_dos': kgrid_dos
    }
    
    with open(WORK_DIR / 'dos_data.json', 'w') as f:
        json.dump(dos_results, f, indent=2)
    
    print(f"DOS data saved to: {WORK_DIR / 'dos_data.json'}")

---

## Summary

In this notebook, we have:

1. ✓ Understood the DOS concept and its relationship to band structure
2. ✓ Performed the complete DOS workflow (SCF → NSCF → dos.x)
3. ✓ Calculated total and projected DOS
4. ✓ Visualized and interpreted the DOS
5. ✓ Analyzed orbital contributions

### Key Observations for Silicon

1. **Clear band gap** in the DOS confirms semiconducting behavior
2. **Valence band**: Mainly Si 3s (lower) and 3p (upper) character
3. **Conduction band**: Mainly Si 3p character with some 3s mixing
4. **8 electrons** in the valence band (4 per Si atom)

### Practical Tips

| Parameter | Recommendation |
|-----------|----------------|
| K-grid for DOS | 2-4× denser than SCF |
| Occupation | Use 'tetrahedra' for semiconductors |
| Energy resolution | DeltaE = 0.01-0.05 eV |

### Next Notebook
→ **08_Summary_and_Exercises.ipynb**: Workshop summary and practice exercises